In [38]:
#imports
import numpy as np
import pandas as pd
import json
import glob
import tensorflow as tf

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


#visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
#load stop words
stop_words = stopwords.words("english")

In [74]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [75]:
data = load_data('../input/ushmm-dn/ushmm_dn.json')['texts']
print(data[0][0:100])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we 


In [76]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)

In [77]:
print(lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [79]:
def gen_words(texts):
    final = []
    for text in texts:
            new = gensim.utils.simple_preprocess(text, deacc=True)
            final.append(new)
    return (final)

In [80]:
data_words = gen_words(lemmatized_texts)

In [81]:
print(data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [82]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[0][0:20])

['name_bear', 'small_town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school', 'public_school', 'morning']


In [83]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 7), (3, 2), (4, 2), (5, 1), (6, 2), (7, 1), (8, 2), (9, 1), (10, 2), (11, 3), (12, 1), (13, 12), (14, 1), (15, 8), (16, 1), (17, 1), (18, 1), (19, 3)]


In [84]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word,
                                            num_topics=15, update_every=1,
                                           chunksize=100, passes=10,
                                           alpha="auto")

In [85]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = 'mmds', R=30)
vis

/opt/conda/lib/python3.10/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_max

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.296672  0.022115       1        1  17.153293
12    -0.307391  0.179270       2        1  16.187791
9     -0.381476 -0.116084       3        1  10.408712
0     -0.144415 -0.335224       4        1  10.305034
10     0.273823 -0.276979       5        1   8.828476
3      0.064450 -0.347485       6        1   8.668096
14     0.133693  0.361508       7        1   8.123433
11    -0.161109 -0.149828       8        1   6.385696
5      0.072774 -0.170274       9        1   6.144608
13    -0.123125  0.372858      10        1   4.034848
8     -0.036751  0.196208      11        1   2.615789
6      0.284349  0.018287      12        1   0.500109
1      0.245048  0.097726      13        1   0.313668
2      0.190604  0.089176      14        1   0.201700
7      0.186196  0.058726      15        1   0.128747, topic_info=             Term         Freq        Total Category  logprob  loglift
133          camp  2356.000000  2356.000000  Default  30.0000  30.0000
892           use  1730.000000  1730.000000  Default  29.0000  29.0000
351        ghetto  2520.000000  2520.000000  Default  28.0000  28.0000
690      remember  1022.000000  1022.000000  Default  27.0000  27.0000
195        course  1304.000000  1304.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
363   grandfather     0.584459   155.877601  Topic15  -6.0309   1.0689
4439     gendarme     0.360243    23.806289  Topic15  -6.5148   2.4641
2566         poem     0.365533    26.436030  Topic15  -6.5003   2.3739
428    impossible     0.324954    17.564687  Topic15  -6.6179   2.6651
1445  grandmother     0.395462   157.405323  Topic15  -6.4216   0.6685

[1012 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.318463      able
0         2  0.026724      able
0         3  0.042313      able
0         4  0.035632      able
0         6  0.004454      able
...     ...       ...       ...
2250      1  0.174702      yell
2250      2  0.807998      yell
4557      1  0.201833  yugoslav
4557      5  0.605499  yugoslav
4557     14  0.100916  yugoslav

[2104 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 13, 10, 1, 11, 4, 15, 12, 6, 14, 9, 7, 2, 3, 8])